## IMDb Data Fetching

We use a custom IMDb-fetching script to enrich the MovieLens movies with additional information that MovieLens does not provide. For each movie, the script uses the imdbID from links.csv to download key fields such as the cast list and plot synopsis. This information is cleaned and saved into a consistent text-file format so it can be reused without fetching again.

We do this because our project relies on actor-based network analysis, and MovieLens does not include actor data. The IMDb information is therefore essential for creating the movie network: each movie is a node, and movies are connected when they share actors. The fetched IMDb metadata forms the foundation for building the network and running the Louvain community detection, and it allows us to later compare communities using genres, audience ratings, and tags.

In [3]:
# -*- coding: utf-8 -*-

import re
import html
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ---------------- Config ----------------
LINKS_CSV   = "Movielens/links.csv"
MOVIES_CSV  = "Movielens/movies.csv"
OUT_DIR     = Path("data/imdb_txt_synopsis_final")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SELECT_BY   = "all"   # "first_n" or "all"
FIRST_N     = 5000

TOP_ACTORS_PER_MOVIE = 20
MAX_WORKERS = 8
REQUEST_TIMEOUT = 20
RETRIES = 3

UA = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

# -------------- Helpers -----------------
def imdb_tt_from_numeric(imdb_numeric: int) -> str:
    return f"tt0{int(imdb_numeric):06d}"

def safe_title_for_filename(title: str) -> str:
    s = (title or "").strip()
    s = s.replace(" ", "_")
    s = re.sub(r'[\\/:*?"<>|]+', "_", s)
    s = re.sub(r"_+", "_", s)
    return s or "untitled"

def clean_imdb_title(raw: str) -> str:
    """
    From <title> like 'Balto (1995) - Reference View - IMDb'
    -> 'Balto'
    """
    t = raw or ""
    t = t.replace(" - IMDb", "")
    t = re.sub(r"\s*-\s*Reference\s*View", "", t, flags=re.I)
    # strip trailing (YYYY)
    name = re.sub(r"\s*\(\d{4}\)\s*$", "", t).strip()
    return name

def build_session():
    s = requests.Session()
    retries = Retry(
        total=RETRIES,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
    )
    adapter = HTTPAdapter(max_retries=retries, pool_connections=MAX_WORKERS, pool_maxsize=MAX_WORKERS)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    s.headers.update(UA)
    return s

# ----------- Parsing /reference -----------
def parse_reference(html_text: str):
    """
    Extract main fields from /reference page:
    returns dict: name, year, ratingValue, directors(list), actors(list)
    (no description, no genres, no duration)
    """
    soup = BeautifulSoup(html_text, "html.parser")
    out = {
        "name": None,
        "year": None,
        "ratingValue": None,
        "directors": [],
        "actors": [],
    }

    # Title & year from <title>
    ttag = soup.find("title")
    if ttag and ttag.text:
        ttxt = ttag.text.strip()
        out["name"] = clean_imdb_title(ttxt)
        m = re.search(r"\((\d{4})\)\s*(?:-|\Z)", ttxt)
        if m:
            out["year"] = m.group(1)

    # Rating value
    rv = soup.find(attrs={"itemprop": "ratingValue"})
    if rv and rv.text.strip():
        out["ratingValue"] = rv.text.strip()

    # Collect /name/nm links beneath headers
    def collect_names_after_header(header_substrs, limit=None):
        names = []
        for hdr in soup.find_all(["h2", "h3", "h4", "h5"]):
            label = hdr.get_text(" ", strip=True).lower()
            if any(sub in label for sub in header_substrs):
                node = hdr
                steps = 0
                while node and steps < 200:
                    node = node.find_next()
                    steps += 1
                    if not node:
                        break
                    if getattr(node, "name", None) in ("h2", "h3", "h4", "h5"):
                        break
                    if node.name == "a" and node.has_attr("href") and "/name/nm" in node["href"]:
                        nm = node.get_text(strip=True)
                        if nm and nm not in names:
                            names.append(nm)
                            if limit and len(names) >= limit:
                                return names
                break
        return names

    out["directors"] = collect_names_after_header(["directed by", "director"], limit=None)
    out["actors"]    = collect_names_after_header(["cast"], limit=TOP_ACTORS_PER_MOVIE)

    return out

# ----------- Parsing /plotsummary (Synopsis) -----------
def parse_synopsis(html_text: str) -> str:
    """
    Extract the long 'Synopsis' section from /plotsummary.
    - take paragraphs under 'Synopsis' header
    - deduplicate repeated paragraphs
    - trim stuff like 'Contribute to this page...'
    """
    soup = BeautifulSoup(html_text, "html.parser")

    def clean_full_text(full: str) -> str:
        # drop trailing boilerplate
        full = full.split("Contribute to this page", 1)[0]
        full = full.split("Contribute to This Page", 1)[0]
        return full.strip()

    # 1) Explicit 'Synopsis' header then paragraphs until next header
    for hdr in soup.find_all(["h2", "h3", "h4", "h5"]):
        lbl = hdr.get_text(" ", strip=True).lower()
        if "synopsis" in lbl:
            pieces = []
            node = hdr
            steps = 0
            while node and steps < 600:
                node = node.find_next()
                steps += 1
                if not node:
                    break
                if getattr(node, "name", None) in ("h2", "h3", "h4", "h5"):
                    break
                if node.name in ("p", "div", "li"):
                    txt = node.get_text(" ", strip=True)
                    if txt:
                        pieces.append(txt)
            # dedupe paragraphs while preserving order
            unique_pieces = list(dict.fromkeys(pieces))
            syn = "\n\n".join(unique_pieces).strip()
            if syn:
                return html.unescape(clean_full_text(syn))

    # 2) Fallback: big block of <p> in the page
    paras = [p.get_text(" ", strip=True) for p in soup.select("p") if p.get_text(strip=True)]
    if paras:
        unique_p = list(dict.fromkeys(paras))
        syn = "\n\n".join(unique_p).strip()
        return html.unescape(clean_full_text(syn))

    return ""

# -------------- Load data & select set --------------
def prepare_selection():
    links = pd.read_csv(LINKS_CSV).dropna(subset=["imdbId"])
    links["imdbId"] = links["imdbId"].astype(int)
    links["tt"] = links["imdbId"].apply(imdb_tt_from_numeric)
    try:
        movies = pd.read_csv(MOVIES_CSV)[["movieId", "title"]]
    except Exception:
        movies = pd.DataFrame(columns=["movieId", "title"])
    df = links.merge(movies, on="movieId", how="left")
    sel = df.head(FIRST_N) if SELECT_BY == "first_n" else df
    return sel

existing = {p.name for p in OUT_DIR.glob("*.txt")}
index_rows = []

def fetch_one(session: requests.Session, row):
    movieId = int(row.movieId)
    tt = row.tt
    ref_url = f"https://www.imdb.com/title/{tt}/reference"
    syn_url = f"https://www.imdb.com/title/{tt}/plotsummary"

    try:
        # /reference (main fields)
        r_ref = session.get(ref_url, timeout=REQUEST_TIMEOUT)
        r_ref.raise_for_status()
        ref_data = parse_reference(r_ref.text)

        # fallback title/year from MovieLens "Title (YYYY)"
        ml_title = row.title if isinstance(row.title, str) else ""
        if not ref_data["name"] and ml_title:
            ref_data["name"] = re.sub(r"\s*\(\d{4}\)\s*$", "", ml_title).strip()
        if not ref_data["year"] and ml_title:
            m = re.search(r"\((\d{4})\)", ml_title)
            if m:
                ref_data["year"] = m.group(1)

        title = ref_data["name"] or tt
        year = ref_data["year"] or ""
        fname = f"{safe_title_for_filename(title)}_{year}.txt" if year else f"{safe_title_for_filename(title)}.txt"
        out_path = OUT_DIR / fname

        # /plotsummary (synopsis)
        synopsis = ""
        try:
            r_syn = session.get(syn_url, timeout=REQUEST_TIMEOUT)
            r_syn.raise_for_status()
            synopsis = parse_synopsis(r_syn.text)
        except Exception:
            synopsis = ""

        # If file exists, skip writing but still record index row
        if out_path.name in existing:
            index_rows.append({
                "movieId": movieId,
                "tt": tt,
                "title": title,
                "year": year,
                "txt_filename": out_path.name,
                "imdb_url": f"https://www.imdb.com/title/{tt}/",
            })
            return ("SKIP", tt, out_path.name)

        # Format fields we KEEP
        directors = ", ".join(ref_data["directors"])
        actors = ", ".join(ref_data["actors"][:TOP_ACTORS_PER_MOVIE])
        ratingValue = ref_data["ratingValue"] or ""

        # Write TXT (no description, just synopsis)
        lines = []
        lines.append(f"movieId: {movieId}")
        lines.append(f"title: {title}")
        lines.append(f"year: {year}")
        lines.append(f"url: https://www.imdb.com/title/{tt}/")
        lines.append(f"ratingValue: {ratingValue}")
        lines.append(f"directors: {directors}")
        lines.append(f"actors: {actors}")
        lines.append("")
        lines.append("synopsis:")
        lines.append(synopsis)

        out_path.write_text("\n".join(lines), encoding="utf-8")
        existing.add(out_path.name)

        index_rows.append({
            "movieId": movieId,
            "tt": tt,
            "title": title,
            "year": year,
            "txt_filename": out_path.name,
            "imdb_url": f"https://www.imdb.com/title/{tt}/",
        })
        return ("OK", tt, out_path.name)
    except Exception as e:
        return ("FAIL", tt, str(e))

def main():
    sel = prepare_selection()
    print(f"Preparing to fetch {len(sel)} movies")
    print("Saving to:", OUT_DIR.resolve())

    session = build_session()
    futs = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        for row in sel.itertuples(index=False):
            futs.append(ex.submit(fetch_one, session, row))

    done = ok = skip = fail = 0
    for fut in as_completed(futs):
        status, tt, info = fut.result()
        done += 1
        if status == "OK":
            ok += 1
            print(f"[{done:>5}/{len(futs)}] OK   {tt} -> {info}")
        elif status == "SKIP":
            skip += 1
            print(f"[{done:>5}/{len(futs)}] SKIP {tt} ({info})")
        else:
            fail += 1
            print(f"[{done:>5}/{len(futs)}] FAIL {tt}: {info}")

    pd.DataFrame(index_rows).to_csv(OUT_DIR / "index_imdb_txt.csv", index=False)
    print("\nDone.")
    print(f"OK: {ok} | SKIP: {skip} | FAIL: {fail}")
    print("Folder:", OUT_DIR.resolve())
    print("Index:", OUT_DIR / "index_imdb_txt.csv")

if __name__ == "__main__":
    main()


Preparing to fetch 9742 movies
Saving to: C:\Users\Notandi\Desktop\Social graph\Lokaverkefni\data\imdb_txt_synopsis_final
[    1/9742] OK   tt0397892 -> Bolt_2008.txt
[    2/9742] FAIL tt0366780: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0366780/reference (Caused by ResponseError('too many 503 error responses'))
[    3/9742] FAIL tt0455824: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0455824/reference/ (Caused by ResponseError('too many 503 error responses'))
[    4/9742] FAIL tt0378109: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0378109/reference (Caused by ResponseError('too many 503 error responses'))
[    5/9742] FAIL tt01031280: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt01031280/reference/ (Caused by ResponseError('too many 503 error responses'))
[    6/9742] FAIL tt0403455: HTTPSConnect

Checking how many movies have a missing synopsis, and 2134 out of 5512 turned out not to have a synopsis listed.

In [3]:
import re
from pathlib import Path

# 
TXT_DIR = Path("data/imdb_txt_synopsis")

placeholder_substring = "it looks like we don't have any synopsis for this title yet"
n_files = 0
n_missing = 0

for path in TXT_DIR.glob("*.txt"):
    text = path.read_text(encoding="utf-8", errors="ignore")
    n_files += 1

    # grab everything after the line "synopsis:"
    m = re.search(r"^synopsis:\s*\n(.*)\Z", text, flags=re.S | re.M)
    synopsis = m.group(1).strip() if m else ""

    # normalize for checking
    low = synopsis.lower()

    # conditions for "missing synopsis"
    if (not synopsis) or (placeholder_substring in low):
        n_missing += 1

print(f"Total TXT files: {n_files}")
print(f"Movies without real synopsis: {n_missing}")
if n_files > 0:
    print(f"Share missing: {n_missing / n_files:.2%}")


Total TXT files: 5512
Movies without real synopsis: 2134
Share missing: 38.72%


In [ ]:
# 

In [1]:
import os
import glob
import random

# Path to your folder
folder = "data/imdb_txt_synopsis"

# Phrase that indicates missing synopsis
missing_phrase = "we don't have any synopsis for this title yet"

# Collect files that have a real synopsis
valid_files = []

for path in glob.glob(os.path.join(folder, "*.txt")):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read().strip()
    
    # Skip empty or boilerplate-only synopses
    if not text:
        continue
    if missing_phrase.lower() in text.lower():
        continue
    # Optional: require minimum length to avoid junk
    if len(text) < 50:
        continue
    
    valid_files.append(path)

print("Number of movies with real synopsis:", len(valid_files))

# If you have more than 1000, sample 1000
N = 3390
if len(valid_files) > N:
    random.seed(42)  # for reproducibility
    sampled_files = random.sample(valid_files, N)
else:
    sampled_files = valid_files  # just use all

print("Movies selected:", len(sampled_files))

# Optionally, extract IDs from filenames and save them to a file
movie_ids = []
for path in sampled_files:
    filename = os.path.basename(path)        # e.g. "tt1234567.txt"
    movie_id = os.path.splitext(filename)[0] # e.g. "tt1234567"
    movie_ids.append(movie_id)

# Save the selected IDs so you can merge with your MovieLens/IMDb data
import csv
with open("selected_movie_ids_1000.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["imdb_id"])
    for mid in movie_ids:
        writer.writerow([mid])


Number of movies with real synopsis: 3390
Movies selected: 3390
